In [ ]:
!pip install protobuf==3.20.0

In [1]:
import sys
sys.path.append("/ibkr")

In [2]:
!pwd

/ibkr/jupyter


In [3]:
import time
import datetime
import pandas as pd
from databaseClass import DB
import passwords
from utils import DataProcessor
from utils import ModelBuilder
from utils import Predictor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn
import pytz
from joblib import dump, load

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
userName = passwords.userName
userPass = passwords.userPass
dataBaseName = passwords.dataBaseName
host = passwords.host

In [5]:
db = DB(userName=userName, userPass=userPass, dataBaseName=dataBaseName, host=host, docker=True)

In [ ]:
2023-06-21 20:57:03.165

In [5]:
now = datetime.datetime(2023, 6, 21, 20, 57, 3, 410027, pytz.UTC) # for testing
five_minutes_ago = now - datetime.timedelta(minutes=5)

In [6]:
df = db.DBtoDF(f"SELECT * FROM tickdata_jun21 WHERE timestamp BETWEEN '{five_minutes_ago}' AND '{now}'")

/ibkr/databaseClass.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=conn)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ticktype   241 non-null    int64         
 1   price      241 non-null    float64       
 2   volume     241 non-null    int64         
 3   timestamp  241 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.7 KB


In [10]:
df = db.DBtoDF(f"SELECT * FROM tickdata_jun21")

/ibkr/databaseClass.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=conn)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ticktype   1582 non-null   int64         
 1   price      1582 non-null   float64       
 2   volume     1582 non-null   int64         
 3   timestamp  1582 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 49.6 KB


In [12]:
df.head()

,ticktype,price,volume,timestamp
0,1,4410.00,0,2023-06-21 20:22:28.671067
1,2,4410.25,0,2023-06-21 20:22:28.689977
2,4,4410.00,0,2023-06-21 20:22:28.696408
3,5,0.00,11,2023-06-21 20:22:28.702679
4,5,0.00,11,2023-06-21 20:22:28.708767


In [ ]:
# Build Model    
    
df['timestamp'] = df['timestamp'].dt.tz_localize('UTC') #adding this to update to utc

# Create a DataProcessor instance
processor = DataProcessor(df) 
# Process the df
processor.process_df()

# Scale and shift the data
scaler, close_scaler = processor.scale_shift_data(look_ahead)
# Create the X and y datasets
X, y = processor.scaled_df, processor.shifted_df['Close']
# Create train test split
X_train, X_test, y_train, y_test = processor.create_train_test_split(X, y)
# Create final train and test datasets
# TIME_STEPS = 60
X_train, y_train = processor.create_dataset(X_train, y_train, TIME_STEPS)
X_test, y_test = processor.create_dataset(X_test, y_test, TIME_STEPS)
# Number of features in the data
n_features = X_train.shape[2]

# Create a ModelBuilder instance and build the model
builder = ModelBuilder(n_features, TIME_STEPS)
# Train the model
model, history = builder.train_model(X_train, y_train, X_test, y_test, epochs=20, batch_size=64)
# Save the model
model_path = 'models/model.h5'
builder.save_model(model_path)
# Plot loss
# builder.plot_loss(history) #this needs to save vs show
# Create a Predictor instance
predictor = Predictor(model, processor)
# Predict
predictions = predictor.predict(look_ahead, TIME_STEPS)
# Rescale predictions
rescaled_predictions = predictor.rescale_prediction(predictions)

In [13]:
df['timestamp'] = df['timestamp'].dt.tz_localize('UTC') #adding this to update to utc

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   ticktype   1582 non-null   int64              
 1   price      1582 non-null   float64            
 2   volume     1582 non-null   int64              
 3   timestamp  1582 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), int64(2)
memory usage: 49.6 KB


In [15]:
df.head()

,ticktype,price,volume,timestamp
0,1,4410.00,0,2023-06-21 20:22:28.671067+00:00
1,2,4410.25,0,2023-06-21 20:22:28.689977+00:00
2,4,4410.00,0,2023-06-21 20:22:28.696408+00:00
3,5,0.00,11,2023-06-21 20:22:28.702679+00:00
4,5,0.00,11,2023-06-21 20:22:28.708767+00:00


In [16]:
# Create a DataProcessor instance
processor = DataProcessor(df) 

In [17]:
processor.process_df()

In [18]:
processor.processed_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3773 entries, 2023-06-21 20:22:28+00:00 to 2023-06-21 21:25:20+00:00
Freq: S
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    3773 non-null   float64
 1   High    3773 non-null   float64
 2   Low     3773 non-null   float64
 3   Close   3773 non-null   float64
 4   Volume  3773 non-null   float64
dtypes: float64(5)
memory usage: 176.9 KB


In [19]:
processor.processed_df.head()

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-21 20:22:28+00:00,4410.0,4410.25,4410.0,4410.00,22.0
2023-06-21 20:22:29+00:00,4410.0,4410.25,4410.0,4410.25,2.0
2023-06-21 20:22:30+00:00,4410.0,4410.25,4410.0,4410.25,0.0
2023-06-21 20:22:31+00:00,4410.0,4410.25,4410.0,4410.25,0.0
2023-06-21 20:22:32+00:00,4410.0,4410.25,4410.0,4410.00,1.0


In [20]:
# Scale and shift the data
scaler, close_scaler = processor.scale_shift_data(look_ahead=5)

In [27]:
processor.scaled_df.head(5)

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-21 20:22:28+00:00,0.8,0.666667,0.8,0.666667,0.115789
2023-06-21 20:22:29+00:00,0.8,0.666667,0.8,0.833333,0.010526
2023-06-21 20:22:30+00:00,0.8,0.666667,0.8,0.833333,0.000000
2023-06-21 20:22:31+00:00,0.8,0.666667,0.8,0.833333,0.000000
2023-06-21 20:22:32+00:00,0.8,0.666667,0.8,0.666667,0.005263


In [26]:
processor.shifted_df.head(5)

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-21 20:22:28+00:00,0.8,0.666667,0.8,0.666667,0.000000
2023-06-21 20:22:29+00:00,0.8,0.666667,0.8,0.833333,0.263158
2023-06-21 20:22:30+00:00,0.8,0.666667,0.8,0.833333,0.000000
2023-06-21 20:22:31+00:00,0.8,0.666667,0.8,0.833333,0.000000
2023-06-21 20:22:32+00:00,0.8,0.666667,0.8,0.833333,0.005263


In [28]:
X, y = processor.scaled_df, processor.shifted_df['Close']

In [29]:
X

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-21 20:22:28+00:00,0.8,0.666667,0.8,0.666667,0.115789
2023-06-21 20:22:29+00:00,0.8,0.666667,0.8,0.833333,0.010526
2023-06-21 20:22:30+00:00,0.8,0.666667,0.8,0.833333,0.000000
2023-06-21 20:22:31+00:00,0.8,0.666667,0.8,0.833333,0.000000
2023-06-21 20:22:32+00:00,0.8,0.666667,0.8,0.666667,0.005263
...,...,...,...,...,...
2023-06-21 21:25:11+00:00,0.6,0.666667,0.6,0.666667,0.000000
2023-06-21 21:25:12+00:00,0.6,0.666667,0.6,0.666667,0.000000
2023-06-21 21:25:13+00:00,0.6,0.666667,0.6,0.666667,0.000000


In [31]:
X_train, X_test, y_train, y_test = processor.create_train_test_split(X, y)

In [34]:
X_train.head()

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-21 20:22:28+00:00,0.8,0.666667,0.8,0.666667,0.115789
2023-06-21 20:22:29+00:00,0.8,0.666667,0.8,0.833333,0.010526
2023-06-21 20:22:30+00:00,0.8,0.666667,0.8,0.833333,0.000000
2023-06-21 20:22:31+00:00,0.8,0.666667,0.8,0.833333,0.000000
2023-06-21 20:22:32+00:00,0.8,0.666667,0.8,0.666667,0.005263


In [36]:
X_train, y_train = processor.create_dataset(X_train, y_train, time_steps=60, look_ahead=5, for_training=True)
X_test, y_test = processor.create_dataset(X_test, y_test, time_steps=60, look_ahead=5, for_training=True)

In [40]:
X_train[0]

array([[0.8       , 0.66666667, 0.8       , 0.66666667, 0.11578947],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.01052632],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.66666667, 0.00526316],
       [0.8       , 0.66666667, 0.8       , 0.66666667, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.26315789],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.00526316],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.05263158],
       [0.8       , 0.66666667, 0.

In [41]:
X_train[1]

array([[0.8       , 0.66666667, 0.8       , 0.83333333, 0.01052632],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.66666667, 0.00526316],
       [0.8       , 0.66666667, 0.8       , 0.66666667, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.26315789],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.00526316],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.05263158],
       [0.8       , 0.66666667, 0.8       , 0.83333333, 0.        ],
       [0.8       , 0.66666667, 0.

In [42]:
n_features = X_train.shape[2]

In [43]:
n_features

5

In [44]:
#### Below for inference

In [ ]:
# Get current time
#now = datetime.datetime.now(pytz.UTC)
now = datetime.datetime(2023, 6, 21, 20, 57, 3, 410027, pytz.UTC) # for testing


# Get the time one minute ago
one_minute_ago = now - datetime.timedelta(minutes=1)

# Get the time five minutes ago
five_minutes_ago = now - datetime.timedelta(minutes=5)

# Query the database for all rows from the last minute
new_data = app.db.DBtoDF(f"SELECT * FROM tickdata_jun21 WHERE timestamp BETWEEN '{five_minutes_ago}' AND '{now}'")
#new_data['timestamp'] = pd.to_datetime(new_data['timestamp'])
new_data['timestamp'] = new_data['timestamp'].dt.tz_localize('UTC')
print("Step 1")
print(new_data.head())


# Process new_data into the proper format for the model
preprocessor.df = new_data
preprocessor.process_df()
print("step 2")
print(preprocessor.processed_df.head())
# After preprocessing and filling, trim data to only last minute
preprocessor.processed_df = preprocessor.processed_df.tail(66)
preprocessor.processed_df.to_csv(f"testdata/{now}.csv")
print("Step 3")
print(preprocessor.processed_df.head())

preprocessor.scale_shift_data(look_ahead=0, for_training=False)

X = preprocessor.scaled_df
print("Step 4")
print(X[0:5])
print(y[0:5])
print(X.shape)
print(y.shape)
#X, y = preprocessor.create_dataset(X, y, time_steps)
X = preprocessor.create_dataset(X, y=None, time_steps=60, look_ahead=0, for_training=False)



print("Step 5")
print(X[0:5])
print(y[0:5])

# Load the model
model_builder.load_model("models/model.h5")

# Assign the model to the predictor
predictor.model = model_builder.model

# Predict the next 'look_ahead' steps
predictions = predictor.predict(look_ahead, time_steps)
print("step 6")
print(predictions[0:5])

# Rescale predictions back to the original scale
rescaled_predictions = predictor.rescale_prediction(predictions)
print("Step 7:")
print(rescaled_predictions[0:5])

# Fetch the last predicted value
price = rescaled_predictions[-1][0]
print("Step 8")
print(price)

print("Step 9")
# print(new_data['price_4'][-1])
if price > preprocessor.processed_df['Close'][-1]:
    # Buy if the predicted price is higher than the current price
    #app.placeOrder(app.nextOrderId(), order, Order(action="BUY", totalQuantity=1, orderType="LMT", lmtPrice=price))
    print(f"expected price: {price} Action: buy, {preprocessor.processed_df['Close'][-1]}")

elif price < preprocessor.processed_df['Close'][-1]:
    # Sell if the predicted price is lower than the current price
    #app.placeOrder(app.nextOrderId(), order, Order(action="SELL", totalQuantity=1, orderType="LMT", lmtPrice=price))
    print(f"expected price: {price} Action: sell, {preprocessor.processed_df['Close'][-1]}")

In [6]:
# Get current time
#now = datetime.datetime.now(pytz.UTC)
now = datetime.datetime(2023, 6, 21, 20, 57, 3, 410027, pytz.UTC) # for testing


# Get the time one minute ago
one_minute_ago = now - datetime.timedelta(minutes=1)

# Get the time five minutes ago
five_minutes_ago = now - datetime.timedelta(minutes=5)

# Query the database for all rows from the last minute
new_data = db.DBtoDF(f"SELECT * FROM tickdata_jun21 WHERE timestamp BETWEEN '{five_minutes_ago}' AND '{now}'")

/ibkr/databaseClass.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=conn)


In [7]:
new_data.head()

,ticktype,price,volume,timestamp
0,5,0.00,1,2023-06-21 20:52:06.636343
1,5,0.00,5,2023-06-21 20:52:08.940320
2,2,4409.75,0,2023-06-21 20:52:08.964836
3,2,4409.50,0,2023-06-21 20:52:09.039789
4,5,0.00,10,2023-06-21 20:52:15.050502


In [28]:
pred_preprocessor = DataProcessor(df=None)

In [29]:
pred_preprocessor.df = new_data

In [30]:
pred_preprocessor.process_df()

In [31]:
pred_preprocessor.processed_df

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-21 20:52:06,NaN,NaN,NaN,NaN,1.0
2023-06-21 20:52:07,NaN,NaN,NaN,NaN,0.0
2023-06-21 20:52:08,NaN,4409.75,4409.75,NaN,5.0
2023-06-21 20:52:09,NaN,4409.50,4409.50,NaN,0.0
2023-06-21 20:52:10,NaN,4409.50,4409.50,NaN,0.0
...,...,...,...,...,...
2023-06-21 20:56:59,4409.25,4409.50,4409.25,4409.5,0.0
2023-06-21 20:57:00,4409.25,4409.50,4409.25,4409.5,0.0
2023-06-21 20:57:01,4409.25,4409.50,4409.25,4409.5,0.0


In [32]:
#pred_preprocessor.processed_df = pred_preprocessor.processed_df.tail(66)
pred_preprocessor.processed_df = pred_preprocessor.processed_df.tail(250)

In [33]:
pred_preprocessor.processed_df

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-21 20:52:54,4409.25,4409.50,4409.25,4409.50,0.0
2023-06-21 20:52:55,4409.25,4409.50,4409.25,4409.50,0.0
2023-06-21 20:52:56,4409.25,4409.50,4409.25,4409.50,0.0
2023-06-21 20:52:57,4409.25,4409.50,4409.25,4409.50,0.0
2023-06-21 20:52:58,4409.25,4409.50,4409.25,4409.25,2.0
...,...,...,...,...,...
2023-06-21 20:56:59,4409.25,4409.50,4409.25,4409.50,0.0
2023-06-21 20:57:00,4409.25,4409.50,4409.25,4409.50,0.0
2023-06-21 20:57:01,4409.25,4409.50,4409.25,4409.50,0.0


In [34]:
pred_preprocessor.scale_shift_data(look_ahead=0, for_training=False)

(MinMaxScaler(), MinMaxScaler())

In [35]:
X = pred_preprocessor.scaled_df

In [36]:
X

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-21 20:52:54,0.333333,0.333333,0.333333,0.50,0.000000
2023-06-21 20:52:55,0.333333,0.333333,0.333333,0.50,0.000000
2023-06-21 20:52:56,0.333333,0.333333,0.333333,0.50,0.000000
2023-06-21 20:52:57,0.333333,0.333333,0.333333,0.50,0.000000
2023-06-21 20:52:58,0.333333,0.333333,0.333333,0.25,0.010526
...,...,...,...,...,...
2023-06-21 20:56:59,0.333333,0.333333,0.333333,0.50,0.000000
2023-06-21 20:57:00,0.333333,0.333333,0.333333,0.50,0.000000
2023-06-21 20:57:01,0.333333,0.333333,0.333333,0.50,0.000000


In [37]:
X = pred_preprocessor.create_dataset(X, y=None, time_steps=60, look_ahead=0, for_training=False)

In [39]:
len(X)

190

In [40]:
model_builder = ModelBuilder(n_features=5, time_steps=60)
predictor = Predictor(model=None, preprocessor=pred_preprocessor)
model_builder.load_model("../models/model.h5")

# Assign the model to the predictor
predictor.model = model_builder.model
# Predict the next 'look_ahead' steps
predictions = predictor.predict(time_steps=60, for_training=False)

6/6 [==============================] - 0s 6ms/step


In [41]:
predictions

array([[0.23812972],
       [0.21817155],
       [0.20020513],
       [0.18480946],
       [0.17214589],
       [0.16210766],
       [0.15443254],
       [0.15274526],
       [0.15054107],
       [0.14865759],
       [0.14707857],
       [0.14578785],
       [0.14475845],
       [0.16335495],
       [0.19343972],
       [0.22959436],
       [0.26691508],
       [0.30200914],
       [0.31963152],
       [0.3251544 ],
       [0.32361722],
       [0.3178501 ],
       [0.310019  ],
       [0.30162185],
       [0.2936183 ],
       [0.29932553],
       [0.31237233],
       [0.33451626],
       [0.3797043 ],
       [0.433237  ],
       [0.48745903],
       [0.5376426 ],
       [0.5860698 ],
       [0.62735194],
       [0.66269106],
       [0.67864853],
       [0.69101894],
       [0.7131095 ],
       [0.7335184 ],
       [0.76511824],
       [0.78544754],
       [0.7983215 ],
       [0.8061627 ],
       [0.81008244],
       [0.811234  ],
       [0.81057084],
       [0.79307663],
       [0.775

In [20]:
predictions

array([[0.34899536],
       [0.35761642],
       [0.3638836 ],
       [0.36821687],
       [0.37101164],
       [0.37262303]], dtype=float32)

In [42]:
rescaled_predictions = predictor.rescale_prediction(predictions)

In [50]:
rescaled_predictions[-1][0]

4409.4727

In [22]:
rescaled_predictions

array([[4409.3374],
       [4409.3394],
       [4409.341 ],
       [4409.3423],
       [4409.343 ],
       [4409.3433]], dtype=float32)

In [46]:
price = rescaled_predictions[-1][0]

In [47]:
price

4409.4727

In [45]:
X.shape

(190, 60, 5)